## Data Augmention

In [7]:
import pandas as pd

df = pd.read_excel('data/raw/new_clean_data_all.xlsx')
print(len(df))

46588


In [8]:
all_labelize_df = []
new_column=['無標註','自殺與憂鬱','無助或無望','正向文字','其他負向文字','生理反應或醫療狀況','自殺行為']
for label in new_column:
    all_labelize_df.append(df[df[label] == 1].loc[:])
for i in all_labelize_df:
    print(len(i))

34599
3443
279
209
7362
557
139


In [9]:
# TODO: split "all_labelize_df" into "test_labelize_df" and "labelize_df"
# random_state = 1
test_labelize_df = []
labelize_df = []
for idx, a_df in enumerate(all_labelize_df):
    n0 = 40
    if idx == 0:
        n0 = 200
    elif idx == 1:
        n0 = 200
    elif idx == 2:
        n0 = 35
    elif idx == 3:
        n0 = 30
    elif idx == 4:
        n0 = 200
    elif idx == 5:
        n0 = 50
    else:
        n0 = 20
    test_labelize_df.append(a_df.sample(n=n0, random_state=12345))
    labelize_df.append(a_df.drop(test_labelize_df[-1].index))
for i in test_labelize_df:
    print(len(i))
for i in labelize_df:
    print(len(i))


200
200
35
30
200
50
20
34399
3243
244
179
7162
507
119


In [10]:
print(labelize_df[0].iloc[0])
print(labelize_df[0].shape)
SMALL_DF = labelize_df[0].iloc[:1]
print((SMALL_DF).shape)
# print(type(labelize_df[0]))
# labelize_df[0].append(SMALL_DF)
print(pd.concat([labelize_df[0], (SMALL_DF)], ignore_index=True).shape)

Unnamed: 0                                         0
No.                                                1
TextID                                    0038f0569e
Title                                       憂鬱症的我...
Sentence      以下文長...如果排版上有問題請見諒:loudly_crying_face:
無標註                                                1
自殺與憂鬱                                              0
無助或無望                                              0
正向文字                                               0
其他負向文字                                             0
生理反應或醫療狀況                                          0
自殺行為                                               0
Name: 0, dtype: object
(34399, 12)
(1, 12)
(34400, 12)


In [11]:
# split Augmentation Data
all_middle_df = labelize_df[0]
print(all_middle_df.shape)
to_add = 2500
Augment_df = []
for i in range(4):
    Augment_df.append(all_middle_df.sample(n=to_add, random_state=12345))
    all_middle_df = all_middle_df.drop(Augment_df[-1].index)
print(all_middle_df.shape)
labelize_df[0] = all_middle_df

for i in range(len(labelize_df)):
    labelize_df[i] = labelize_df[i].reset_index(drop=True)
# for df in labelize_df:
#     print(df)
for i in range(len(Augment_df)):
    Augment_df[i] = Augment_df[i].reset_index(drop=True)
for df in Augment_df:
    print(df.shape)
to_train = 3000
training_df = []

augment_id = 0
for idx, a_df in enumerate(labelize_df):
    augment_label = new_column[idx]
    if a_df.shape[0] > to_train:
        training_df.append(a_df.sample(n=to_train, random_state=12345))
        print("augment: ",training_df[-1].shape)
    else:
        to_augment_df = a_df
        boundary = a_df.shape[0]
        for i in range(to_add):
            # try: 
                # print(idx, i)
                if len( str(Augment_df[augment_id].at[i, 'Sentence'])) < 5:
                    augment_sentence = str(a_df.at[i % boundary, 'Sentence']) + str(Augment_df[augment_id].at[i, 'Sentence'])
                else:
                    augment_sentence = str(a_df.at[i % boundary, 'Sentence']) + str(Augment_df[augment_id].at[i, 'Sentence'])[:5]
                to_augment_df = pd.concat([to_augment_df, a_df.iloc[i % boundary:(i % boundary)+1]], ignore_index=True)
                to_augment_df.at[boundary+i, 'Sentence'] = augment_sentence
            # except:
            #     print(i % boundary)
        training_df.append(to_augment_df)
        print("augment: ",to_augment_df.shape)
        augment_id += 1
for df in training_df:
    print(df.shape)

(34399, 12)
(24399, 12)
(2500, 12)
(2500, 12)
(2500, 12)
(2500, 12)
augment:  (3000, 12)
augment:  (3000, 12)
augment:  (2744, 12)
augment:  (2679, 12)
augment:  (3000, 12)
augment:  (3007, 12)
augment:  (2619, 12)
(3000, 12)
(3000, 12)
(2744, 12)
(2679, 12)
(3000, 12)
(3007, 12)
(2619, 12)


In [12]:
augmented_df = pd.concat(training_df,axis=0, ignore_index=True).drop(['Unnamed: 0'], axis=1)
print(augmented_df.columns)
augmented_df.to_excel('data/raw/augmented_by_categories_0525_v2.xlsx')
test_df = pd.concat(test_labelize_df,axis=0, ignore_index=True)
print(test_df.columns)
test_df.to_excel('data/raw/augmented_test_0525_v2.xlsx')

Index(['No.', 'TextID', 'Title', 'Sentence', '無標註', '自殺與憂鬱', '無助或無望', '正向文字',
       '其他負向文字', '生理反應或醫療狀況', '自殺行為'],
      dtype='object')
Index(['Unnamed: 0', 'No.', 'TextID', 'Title', 'Sentence', '無標註', '自殺與憂鬱',
       '無助或無望', '正向文字', '其他負向文字', '生理反應或醫療狀況', '自殺行為'],
      dtype='object')


In [4]:
# # 4/7
# # 數量比較少的['無助或無望','正向文字','生理反應或醫療狀況','自殺行為']先各補1000句看看觀察效果
# to_add_class = ['無助或無望','正向文字','生理反應或醫療狀況','自殺行為']
# to_add = 500
# random_neutral_df = labelize_df[0].sample(n=to_add*4, random_state=1).reset_index()
# print(random_neutral_df.columns)
# wrong_number = []
# for i in range(len(to_add_class)):
#     k = 0
#     if i == 0:
#         k = 2
#     elif i == 1:
#         k = 3
#     elif i == 2:
#         k = 5
#     else:
#         k = 6
#     for j in range(to_add):
#         random_neutral_df.at[i*to_add+j, 'Sentence'] = str(random_neutral_df.at[i*to_add+j, 'Sentence']) + str(labelize_df[k].sample(n=1, random_state = i*to_add+j).reset_index().at[0,'Sentence'])
#         random_neutral_df.at[i*to_add+j, to_add_class[i]] = 1
#         random_neutral_df.at[i*to_add+j, '無標註'] = 0
#     labelize_df[k] = pd.concat([labelize_df[k], random_neutral_df[i*to_add:(i+1)*to_add]], axis=0, ignore_index=True)

Index(['index', 'Unnamed: 0', 'No.', 'TextID', 'Title', 'Sentence', '無標註',
       '自殺與憂鬱', '無助或無望', '正向文字', '其他負向文字', '生理反應或醫療狀況', '自殺行為'],
      dtype='object')


In [5]:
# for i in labelize_df:
#     print(len(i))

34399
3243
744
684
7162
1007
619


In [6]:
# partial_augment_df = []
# for a_df in labelize_df:
#     if len(a_df) > 800:
#         partial_augment_df.append(a_df.sample(n=800, random_state=1))
#     else:
#         partial_augment_df.append(a_df)
# for i in partial_augment_df:
#     print(len(i))


800
800
744
684
800
800
619


In [7]:
# augmented_df = pd.concat(partial_augment_df,axis=0, ignore_index=True).drop(['index'], axis=1)
# print(augmented_df.columns)

Index(['Unnamed: 0', 'No.', 'TextID', 'Title', 'Sentence', '無標註', '自殺與憂鬱',
       '無助或無望', '正向文字', '其他負向文字', '生理反應或醫療狀況', '自殺行為'],
      dtype='object')


In [8]:
augmented_df.to_excel('data/raw/new_clean_data_augmented_split_v2.xlsx')

In [11]:
test_df = pd.concat(test_labelize_df,axis=0, ignore_index=True)
print(test_df.columns)
test_df.to_excel('data/raw/augmented_test_0428_v1.xlsx')

In [ ]:
reduce_augmented_df = pd.read_excel('new_clean_data_augmented.xlsx')


## Reduce Category

In [1]:
import pandas as pd
combine_df = pd.read_excel('new_clean_data_all.xlsx')

for i in range(len(combine_df)):
    if combine_df['其他負向文字'].iloc[i] == 1:
        combine_df.at[i, '自殺與憂鬱'] = 1

In [2]:
all_labelize_df = []
new_column=['無標註','自殺與憂鬱','無助或無望','正向文字','其他負向文字','生理反應或醫療狀況','自殺行為']
for label in new_column:
    if label != '其他負向文字':
        all_labelize_df.append(combine_df[combine_df[label] == 1].loc[:])
for i in all_labelize_df:
    print(len(i))

34599
10805
279
209
557
139


In [3]:
# 數量比較少的['無助或無望','正向文字','生理反應或醫療狀況','自殺行為']先各補1000句看看觀察效果
to_add_class = ['無助或無望','正向文字','生理反應或醫療狀況','自殺行為']
to_add = 2000
random_neutral_df = all_labelize_df[0].sample(n=to_add*4, random_state=1).reset_index()
print(random_neutral_df.columns)
wrong_number = []
for i in range(len(to_add_class)):
    k = 0
    if i == 0:
        k = 2
    elif i == 1:
        k = 3
    elif i == 2:
        k = 4
    else:
        k = 5
    for j in range(to_add):
        random_neutral_df.at[i*to_add+j, 'Sentence'] = str(random_neutral_df.at[i*to_add+j, 'Sentence']) + str(all_labelize_df[k].sample(n=1, random_state = i*to_add+j).reset_index().at[0,'Sentence'])
        random_neutral_df.at[i*to_add+j, to_add_class[i]] = 1
        random_neutral_df.at[i*to_add+j, '無標註'] = 0
    all_labelize_df[k] = pd.concat([all_labelize_df[k], random_neutral_df[i*to_add:(i+1)*to_add]], axis=0, ignore_index=True)

Index(['index', 'Unnamed: 0', 'No.', 'TextID', 'Title', 'Sentence', '無標註',
       '自殺與憂鬱', '無助或無望', '正向文字', '其他負向文字', '生理反應或醫療狀況', '自殺行為'],
      dtype='object')


In [4]:
partial_augment_df = []
for a_df in all_labelize_df:
    if len(a_df) > 2500:
        partial_augment_df.append(a_df.sample(n=2500, random_state=1))
    else:
        partial_augment_df.append(a_df)
for i in partial_augment_df:
    print(len(i))

2500
2500
2279
2209
2500
2139


In [5]:
augmented_df = pd.concat(partial_augment_df,axis=0, ignore_index=True).drop(['index','其他負向文字'], axis=1)
print(augmented_df.columns)

Index(['Unnamed: 0', 'No.', 'TextID', 'Title', 'Sentence', '無標註', '自殺與憂鬱',
       '無助或無望', '正向文字', '生理反應或醫療狀況', '自殺行為'],
      dtype='object')


In [9]:
augmented_df.to_excel('new_clean_data_augmented_reduce.xlsx')